tgb - 6/25/2020 - Investigate NN stability as a function of its number of layers to test Vladimir's hypothesis

# Imports

In [4]:
import sys
sys.path.insert(1,"/home1/07064/tg863631/anaconda3/envs/CbrainCustomLayer/lib/python3.6/site-packages") #work around for h5py
from cbrain.imports import *
from cbrain.cam_constants import *
from cbrain.normalization import *
from cbrain.utils import *
from cbrain.layers import *
from cbrain.data_generator import DataGenerator

import json
import math
import matplotlib as mpl
import matplotlib.pylab as pl
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

import scipy.integrate as sin

import tensorflow as tf
from tensorflow import keras as K
from tensorflow import math as tfm
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
import tensorflow_probability as tfp
from tensorflow.python.ops.parallel_for.gradients import batch_jacobian

# Load uwnet utilities (assumes a local symlink to uwnet utilities)
# from uwnet.wave import *
# from uwnet.spectra import *
# from uwnet.plots.jacobian import plot

from pylab import *
import numpy
import pickle
import xarray

# Read json and h5 files from Jordan's 32col search

Use the standard cbrain conda environment for this part

Identify four networks to test (4,6,8,10 layers all with 512 nodes):  
- 4 layers is trial-id 75 in Batch 2 TT 31  
- 6 layers is trial-id 18 in Batch 1 TT 18  
- 8 layers is trial-id 12 in Batch 1 TT 12  
- 10 layers is trial-id 58 in Batch 2 TT 14

In [12]:
path32 = '/oasis/projects/nsf/sio134/jott1/32colModels/'

In [13]:
Layers = [4,6,8,10]
pathLay = ['Batch2/00031.','Batch1/00018.','Batch1/00012.','Batch2/00014.']

In [38]:
pathUW = '/oasis/projects/nsf/sio134/tbeucler/GITHUB/CBRAIN-CAM/notebooks/tbeucler_devlog/UW_DATA/'

In [42]:
for ilay,path in enumerate(pathLay):
    print('nb layer=',str(Layers[ilay]),'path=',path)
    NN = load_model(path32+path+'h5')
    save_model(NN,pathUW+'Noah32col_'+str(Layers[ilay])+'layers.h5')

nb layer= 4 path= Batch2/00031.
nb layer= 6 path= Batch1/00018.
nb layer= 8 path= Batch1/00012.
nb layer= 10 path= Batch2/00014.


# Calculate LRF for each NN

Following [https://github.com/tbeucler/CBRAIN-CAM/blob/master/notebooks/tbeucler_devlog/021_Jacobians_for_Noah.ipynb]

Define input and output transform for Jordan's NNs

In [2]:
path_norm = '/oasis/projects/nsf/sio134/jott1/32col/full_physics_essentials_train_month01-06_norm.nc'

In [6]:
in_vars = ['QBP', 'TBP', 'VBP', 'PS', 'SOLIN', 'SHFLX', 'LHFLX']
out_vars = ['PHQ', 'TPHYSTND', 'FSNT', 'FSNS', 'FLNT', 'FLNS', 'PRECT']

In [15]:
path = '/home1/05823/tg851228/NC_DATA/2019-05-01-tropical_mean.nc'
noah_ds = xr.open_mfdataset(path, decode_times=False, concat_dim='time')

OSError: no files to open

# Make stability diagram for each model

Use the UW conda environment for this part